In [36]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta

In [37]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)

def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


In [38]:
def get_new_coins(timestamp_dt, data_table, symbol_col, name_col, source):
    if symbol_col == name_col:
        group_cols = symbol_col
    else:
        group_cols = f"{symbol_col},{name_col}"
    sql = f"""SELECT {symbol_col}, {name_col}, min(timestamp_dt) from 
            `{data_table}` 
            where `timestamp_dt` = FROM_UNIXTIME({timestamp_dt}) 
                and concat({symbol_col}, {name_col}) not in (
                    select concat({symbol_col}, {name_col}) 
                    from {data_table} 
                    where `timestamp_dt` < FROM_UNIXTIME({timestamp_dt})
                )
            group by {group_cols};"""
    print(sql)
    return pd.read_sql(sql, __connect__())
    

In [46]:
today = datetime.now()
today = datetime(today.year, today.month, today.day) + timedelta(days = -2)

In [47]:
today

datetime.datetime(2021, 6, 26, 0, 0)

In [50]:
get_new_coins(today.timestamp(), 'tt_coin_metrics_v2', 'name', 'name', 'tt')

SELECT name, name, min(timestamp_dt) from 
            `tt_coin_metrics_v2` 
            where `timestamp_dt` = FROM_UNIXTIME(1624665600.0) 
                and concat(name, name) not in (
                    select concat(name, name) 
                    from tt_coin_metrics_v2 
                    where `timestamp_dt` < FROM_UNIXTIME(1624665600.0)
                )
            group by name;


,name,name,min(timestamp_dt)
